In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [64]:
# 1 - Load the dataset
rdd = sc.textFile("GBvideos.csv")
rddRows = rdd.map(lambda line: line.split(","))
# remove the headrs row
header = rddRows.first()
rddData= rddRows.filter(lambda line: line != header).filter(lambda line: len(line)==16)

In [65]:
# 2 - Transform the data into RDD pair
tupleData=rddData.map(lambda x:(x[0],list(x[1:])))


In [66]:
# 5(Part a) - sort the data by the Views index, for task 5
def lineViews(line):
    return line[7]
    
rddData = rddData.sortBy(lineViews)

In [67]:
# 3 a - count the distinct values on 5 columns : 
#channel_title
#category_id
#tags
#views
#likes

def specific_column_str(line,index):
    field = line[index]
    return field

def specific_column_int(line,index):
    field = int(line[index])
    return field

channel_title = rddData.map(lambda line : specific_column_str(line,3))
channel_title_distinct = channel_title.distinct()
print('channel_title_distinct count is :' + str(channel_title_distinct.count()))

category_id = rddData.map(lambda line : specific_column_str(line,4))
category_id_distinct = category_id.distinct()
print('category_id_distinct count is :' + str(category_id_distinct.count()))

tags = rddData.map(lambda line : specific_column_str(line,6))
tags_distinct = tags.distinct()
print('tags_distinct count is :' + str(tags_distinct .count()))

views = rddData.map(lambda line : specific_column_int(line,7))
views_distinct = views.distinct()
print('views_distinct count is :' + str(views_distinct .count()))

likes = rddData.map(lambda line : specific_column_int(line,8))
likes_distinct = likes.distinct()
print('likes_distinct count is :' + str(likes_distinct.count()))

channel_title_distinct count is :673
category_id_distinct count is :16
tags_distinct count is :915
views_distinct count is :12195
likes_distinct count is :11093


In [70]:
# 7 - Normalize views and likes values
# We do it here to use it becuase it usfull for for the histogram
viewsMax = views.max()
viewsMin = views.min()
views = views.map(lambda view : (view-viewsMin)/(viewsMax-viewsMin))

likesMax = likes.max()
likesMin = likes.min()
likes = likes.map(lambda like : (like-likesMin)/(likesMax-likesMin))



In [76]:
# 3 b - After we normalize the data to valued from 0 to 1 the histogram function seperate the data to 5
# bins, each of the in size of 0.2 .
print(views.histogram(5))
print(likes.histogram(5))

#Explention of the result:
#in both of the likes and views we have one value that is significanty bigger then the others, which 
#made most of the data to belong to the first bin

([0.0, 0.2, 0.4, 0.6000000000000001, 0.8, 1], [12277, 0, 0, 0, 1])
([0.0, 0.2, 0.4, 0.6000000000000001, 0.8, 1], [12277, 0, 0, 0, 1])


In [11]:
# 4  - Fill in bad values into the mean of the values.
# Here we are working on likes and views of many trending you tube video
# so i think the mean value here fit best.

def changeZeroTOMean(value, meanValue):
    if value == 0:
        return meanValue
    else:
        return value
    
likesMean = likes.mean()
likes = likes.map(lambda line: changeZeroTOMean(line,likesMean))

viewsMean = views.mean()
views = views.map(lambda line: changeZeroTOMean(line,viewsMean))


In [13]:
# 5  (Part b) - Before we sorted all the data by Views.
#The data we work on is you tube videos, and this data usually tested  by views.
#After we sort the data we give the categorical values numerical values when the loest value given to the 
#category with the less views.

titleDict={}
tagsDict={}
def getNumValueTitle(category):
    if category not in titleDict:
        titleDict[category] = len(titleDict)
    return titleDict[category] 
def getNumValueTags(category):
    if category not in tagsDict:
        tagsDict[category] = len(tagsDict)
    return tagsDict[category] 

    

channel_title_numerical = channel_title.map(lambda title:getNumValueTitle(title))
channel_title_numerical.take(3)

tags_numerical = tags.map(lambda tags:getNumValueTags(tags))


In [14]:
# 6 - Here we create from each timestamp, that represent the publish time of the video,
# a RDD of datetime, and from it we create RDD to the hours the days and the months.
from datetime import datetime
timestamp = rddData.map(lambda line : specific_column_str(line,5))

dates = timestamp.map(lambda timestamp:datetime.strptime(timestamp,'%Y-%m-%dT%H:%M:%S.%fZ'))
hours = dates.map(lambda date: date.hour)
days = dates.map(lambda date: date.day)
months = dates.map(lambda date: date.month)



In [55]:
# 8  a - Here we tokenize the 'description' column from the data.
#To do it we used Tokenizer Which is tool in Spark ML 
from pyspark.ml.feature import Tokenizer

descriptionsDF = rddRows.map(lambda line : specific_column_str(line,15)).map(lambda x: x.split(" ")).toDF(["description"])
tokenizer = Tokenizer(inputCol="description", outputCol="words")
tokenized = tokenizer.transform(descriptionsDF)


In [56]:
# 8  b - Here we removed all the stop words from the data.
#To do is we used StopWordsRemover which is tool in Spark ML
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
remover.transform(tokenized)

DataFrame[description: string, words: array<string>, filtered: array<string>]